In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [16]:
def cosine_sim(v1,array_of_vecs):
    cos_sim = []

    v1 = np.array(v1)
    if len(np.shape(array_of_vecs)) == 1:
        array_of_vecs = [array_of_vecs]
    for v2 in array_of_vecs:
        v2 = np.array(v2)
        dot = (np.dot(v1,v2))
        norm_1 = np.linalg.norm(v1)
        norm_2 = np.linalg.norm(v2)
        c = dot/(norm_1*norm_2)
        cos_sim.append(c)
    return cos_sim

In [24]:
def euclidean_dist(v1,array_of_vecs):
    eucl_dist=[]
    
    v1 = np.array(v1)

    if len(np.shape(array_of_vecs)) == 1:
        array_of_vecs = [array_of_vecs]

    for v2 in array_of_vecs:
        v2 = np.array(v2)
        if v1.shape!=v2.shape:
            raise ValueError(f"Shapes don't match: v1 shape: {v1.shape} and v2 shape: {v2.shape}")
            
        d = np.sqrt(np.sum((v1-v2)**2))
        eucl_dist.append(d)
    return eucl_dist

In [27]:
v1 = [1,2]
v2 = [1,1]
array_v = [[3, 2], [5, 6]]
cos_v1_v2 = cosine_sim(v1,v2)
cos_v1_array_v = cosine_sim(v1,array_v)
eucl_v1_v2 = euclidean_dist(v1,v2)
eucl_v1_array_v = euclidean_dist(v1,array_v)
cos_v1_v2
cos_v1_array_v
eucl_v1_v2
eucl_v1_array_v

[2.0, 5.656854249492381]

**Embedding Model**

In [28]:
from sentence_transformers import SentenceTransformer

2025-08-04 16:07:15.273518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754323635.523058      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754323635.593352      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [29]:
model_name = "BAAI/bge-base-en-v1.5"
model = SentenceTransformer(model_name)
model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [32]:
res = model.encode("RAG is awesome")
res.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(768,)

In [33]:
model.encode(['apple', 'car'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([[-0.02102228,  0.06316213,  0.0010086 , ..., -0.00027874,
         0.00038426, -0.0281808 ],
       [ 0.00472379,  0.0158352 ,  0.04418225, ..., -0.05767501,
         0.04269496, -0.00054624]], dtype=float32)

In [34]:
words = ['apple', 'car', 'fruit', 'automobile', 'love', 'sentiment']
vectorized_words = model.encode(words)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [45]:
word = "apple"
print(f"{word}:")
vec_word = vectorized_words[words.index(word)]
print("Cosine Similarities:")
for i,w in enumerate(words):
    print(f"\t{w}: {cosine_sim(vec_word,vectorized_words[i])[0]:.4f}")

print("\n")
print("Euclidean Distances:")

for i,w in enumerate(words):
    print(f"\t{w}: {euclidean_dist(vec_word,vectorized_words[i])[0]:.4f}")

apple:
Cosine Similarities:
	apple: 1.0000
	car: 0.5749
	fruit: 0.7461
	automobile: 0.6485
	love: 0.5540
	sentiment: 0.5020


Euclidean Distances:
	apple: 0.0000
	car: 0.9221
	fruit: 0.7126
	automobile: 0.8384
	love: 0.9445
	sentiment: 0.9980


In [46]:
def retrieve_rel(query, docs, metric='cosine_similarity'):
    query_emb = model.encode(query)
    docs_emb = model.encode(docs)

    if metric=='cosine_similarity':
        dists = cosine_sim(query_emb,docs_emb)
        vals = [(doc, dist) for doc,dist in zip(docs,dists)]
        vals.sort(reverse=True, key=lambda x:x[1])

    elif metric=='euclidean':
        dists = euclidean_dist(query_emb,docs_emb)
        vals = [(doc, dist) for doc,dist in zip(docs,dists)]
        vals.sort(key=lambda x:x[1])
    return vals

In [53]:
docs = [
    "Mt. Fuji is a breathtaking place to explore during autumn.",
    "Santorini offers stunning views to admire during spring.",
    "Banff National Park is a picturesque destination to visit in the summer.",
    "The Great Wall of China is a spectacular site to experience during winter.",
    "The fjords of Norway are a magical place to cruise through in the spring.",
    "Prague is an enchanting city to wander through in winter.",
    "Kyoto's cherry blossoms create a beautiful scene to witness during spring.",
    "Marrakech offers vibrant markets and culture to enjoy in the fall.",
    "The Maldives are a paradisiacal getaway to savor during summer.",
    "The Christmas markets in Vienna are a festive delight to explore in winter."
]

query = "Suggest to me great places to visit in Asia."

In [54]:
score = retrieve_rel(query,docs)
score

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[('The Great Wall of China is a spectacular site to experience during winter.',
  0.60805196),
 ('Mt. Fuji is a breathtaking place to explore during autumn.', 0.5821491),
 ('The Maldives are a paradisiacal getaway to savor during summer.',
  0.56046486),
 ('Santorini offers stunning views to admire during spring.', 0.55122185),
 ('Banff National Park is a picturesque destination to visit in the summer.',
  0.522037),
 ("Kyoto's cherry blossoms create a beautiful scene to witness during spring.",
  0.5150064),
 ('The fjords of Norway are a magical place to cruise through in the spring.',
  0.48470336),
 ('The Christmas markets in Vienna are a festive delight to explore in winter.',
  0.47109163),
 ('Marrakech offers vibrant markets and culture to enjoy in the fall.',
  0.46744198),
 ('Prague is an enchanting city to wander through in winter.', 0.4483725)]

In [56]:
score = retrieve_rel(query, docs, metric='euclidean')
score

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[('The Great Wall of China is a spectacular site to experience during winter.',
  0.88537914),
 ('Mt. Fuji is a breathtaking place to explore during autumn.', 0.91416734),
 ('The Maldives are a paradisiacal getaway to savor during summer.',
  0.9375875),
 ('Santorini offers stunning views to admire during spring.', 0.94739455),
 ('Banff National Park is a picturesque destination to visit in the summer.',
  0.9777147),
 ("Kyoto's cherry blossoms create a beautiful scene to witness during spring.",
  0.9848793),
 ('The fjords of Norway are a magical place to cruise through in the spring.',
  1.0151813),
 ('The Christmas markets in Vienna are a festive delight to explore in winter.',
  1.0285022),
 ('Marrakech offers vibrant markets and culture to enjoy in the fall.',
  1.0320445),
 ('Prague is an enchanting city to wander through in winter.', 1.0503595)]

In [61]:
big_txt = open("/kaggle/input/large-text/large_text.txt").read()

In [63]:
len(big_txt)

3955

In [95]:
big_emb = model.encode(big_txt)
big_emb_few_chars = model.encode(big_txt[:2510])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [96]:
np.array_equal(big_emb,big_emb_few_chars)

True